<h1> N-Able shaped for Datto Comparison - Devices View - Clean / Shape / Compare / Export </h1>

### Import Modules, Create env Variables

In [ ]:
# data import and file manipulation
import os

#data conditioning
import pandas as pd
import re
import datetime as dt
import numpy as np

#data visualization
[REDACTED]/.pyplot as plt
import seaborn as sns

In [ ]:
# add current timestamp to filename for reference
current_time = (dt.datetime.utcnow().strftime('%Y_%m_%d_%H%M%S'))

# git repo folder
git_folder = 'd:/git/example_infrastructure_data_dev'

# dictionary Directory
dictionary_dir = 'd:/git/example_infrastructure_data_dev/dictionaries'

# export folder will contain all csv exported DataFrames for Ticket Creation
export_folder = 'd:/exports/'

In [ ]:
source_dir = 'D:/project_docs/abc_nable_migration/abc_nable_exports/devices'

export_dir = 'D:/project_docs/abc_nable_migration/raw_exports'

In [ ]:
source_files = []
for root, dirs, files in os.walk(source_dir):
    for file in files:
        source_files.append(f"{root}/{file}")

In [ ]:
with open(source_files[0]) as file:
    string = file.read()


In [ ]:
"""

source_files:

['[REDACTED]/.csv',
 '[REDACTED]/.csv',
 '[REDACTED]/.csv',
 '[REDACTED]/.csv']

 """

# Create Device Report
## Deconstruct N-Able Devices CSV Report

In [ ]:
df_nable_devices = pd.read_csv(source_files[0],skiprows=9)

# Data Shaping

## Standardize Column Names

In [ ]:
df = pd.read_csv(f'{dictionary_dir}/standard_column_naming.dict')
column_rename_dict = {}
for index, row in df.iterrows():
    currentColumn = row['currentColumn']
    standardColumn = row['standardColumn']
    column_rename_dict[currentColumn] = standardColumn

In [ ]:
def column_names(string):
    for k, v in column_rename_dict.items():
        try:
            result = re.sub(k.lower(), v, string.lower())
            # print(f'comparing {k.lower()} with {string.lower()}')
            if result != string.lower():
                print(f'Keyword found: {k}')
                print(f'Replacment value: {v}')
                print('\n')
                return v
                break
            elif k.lower() == string.lower():
                print(f'Keyword found: {k}')
                print(f'Replacment value: {v}')
                print('\n')
                return v
                break
        except Exception as e:
            print(e)
            break
    return string

In [ ]:
for column in list(df_nable_devices.columns):
    result = column_names(column)
    df_nable_devices.rename({column:result},axis=1,inplace=True)

## Standardize Client Names

In [ ]:
cu_dict = {'Federal Credit Union':'FCU','Credit Union':'CU'}
def reword_creditunion(string):
    for k, v in cu_dict.items():
        result = re.sub(k,v,string)
        if result != string:
            return (result)
            break
    return string

In [ ]:
df_nable_devices['clientName'] = df_nable_devices['clientName'].apply(reword_creditunion)

In [ ]:
df_nable_devices

# Create and Merge Lan Devices

In [ ]:
df_nable_lan_devices = pd.read_csv(source_files[2],skiprows=3)
df_nable_lan_devices.rename({'Name':'hostname'},inplace=True,axis=1)

In [ ]:
df_nable_lan_devices = df_nable_lan_devices[['hostname','Status','Device Class','Logged in User','Patch Management','Maintenance Window','Microsoft Intune']]

In [ ]:
df_nable_lan_devices

In [ ]:
df_total = pd.merge(df_nable_lan_devices,df_nable_devices,how='outer',on='hostname')

In [ ]:
df_total

In [ ]:
def in_azuread(string):
    try:
        result = re.match(r'azure',string.lower())
        if result:
            return 1
        else:
            return 0
    except:
        return 0

In [ ]:
df_total['inAzureAD'] = df_total['Logged in User'].apply(in_azuread)

In [ ]:
device_type_dict = {'server':'server','workstations':'desktop','laptop':'laptop','switch/router':'network'}
def find_device_type(string):
    for k, v in device_type_dict.items():
        result = re.search(k,string.lower())
        if result:
            return (v)
            break
    return 'unknown'

In [ ]:
df_total['deviceType'] = df_total['Device Class'].apply(find_device_type)
df_total.drop('Device Class',axis=1,inplace=True)

# Create and Merge Warranty Info

In [ ]:
df_warranty = pd.read_csv(source_files[3],skiprows=16,on_bad_lines='warn',delimiter=',',names=['Customer','Site','Device Name','Network Address','Make and Model','Serial Number','Warranty Expiry Date','Warranty Days Left'])

In [ ]:
df_warranty.drop(df_warranty[df_warranty['Customer'] == 'Customer'].index,inplace=True)

In [ ]:
df_warranty = df_warranty[['Device Name','Make and Model','Serial Number','Warranty Expiry Date']]

In [ ]:
df_warranty.rename({'Device Name':'hostname'},axis=1,inplace=True)

In [ ]:
df_total = pd.merge(df_warranty,df_total,how='outer',on='hostname')

# Create Calculated Columns

In [ ]:
def is_intune(string):
    if string == 'Microsoft Intune':
        return 1
    else:
        return 0

In [ ]:
df_total['inIntune'] = df_total['Microsoft Intune'].apply(is_intune)

In [ ]:
for column in list(df_total.columns):
    result = column_names(column)
    df_total.rename({column:result},axis=1,inplace=True)

In [ ]:
df_total

In [ ]:
def isWarrantyExpired(date):
    try:
        warrantyDate = dt.datetime.strptime(date,'%Y-%m-%d')
        timedelta = dt.datetime.now() - warrantyDate
        if timedelta.days > 0:
            return 1
        else:
            return 0
    except Exception as e:
        print(e)
        return 2


In [ ]:
df_total['warrantyDate'].apply(isWarrantyExpired)

In [ ]:
df_total['isWarrantyExpired'] = df_total['warrantyDate'].apply(isWarrantyExpired)

In [ ]:
df_total.drop('msPatchPolicy',axis=1, inplace = True)

In [ ]:

for root, dirs, files in os.walk('D:\exports\patch_policy_scope_exports'):
    merge_exists = False
    for file in files:
        print(merge_exists)
        root_loc = re.sub(r'\\','/',root)
        file_loc = (f"{root_loc}/{file}")
        policy_name = re.findall(r'([^\.]+)[REDACTED]/.csv',file)
        print(f'Attempting to read file loc: {file_loc}')
        print(policy_name)
        df = pd.read_csv(file_loc)
        df['policy'] = policy_name[0]
        while(merge_exists == False):
            df_merged = df
            merge_exists = True
        df_merged = pd.concat([df, df_merged],ignore_index=True)

In [ ]:
df_merged = df_merged[['Customer','Site','Device Name','Network Address','Device Class','Operating System','policy']]

In [ ]:
df_policy = df_merged[['Device Name','policy']]
df_policy.rename({'Device Name':'hostname','policy':'msPatchPolicy'}, axis = 1, inplace=True)

In [ ]:
df_policy

In [ ]:
df_total = pd.merge(df_policy, df_total, how='outer', on='hostname')

In [ ]:
df_total['msPatchPolicy'].fillna('No Policy', inplace=True)

In [ ]:
df_total['operatingSystem'].fillna('Unknown', inplace=True)

In [ ]:
df_windows = df_total[df_total['operatingSystem'].str.contains('Microsoft')]

In [ ]:
def convert_tofloat_with_errors(floatvalue):
    try:
        return int(floatvalue)
    except Exception as e:
        return int(0)

In [ ]:
df_total['inAzureAD'] = df_total['inAzureAD'].apply(convert_tofloat_with_errors)
df_total['monitoredServicesCount'] = df_total['monitoredServicesCount'].apply(convert_tofloat_with_errors)
df_total['inIntune'] = df_total['inIntune'].apply(convert_tofloat_with_errors)

## Add Report Creation Date Info Column

In [ ]:
report_creation_date = (dt.datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S'))
df_total['reportCreationDate'] = report_creation_date

In [ ]:
df_total.to_csv(export_folder + 'nable_device_merged_all_columns_' + str(current_time) + '[REDACTED]/.csv',index=False)

In [ ]:
df_windows[['clientName','hostname','msPatchPolicy']].value_counts(['clientName','msPatchPolicy']).sort_index(ascending=True).to_csv(export_folder + 'nable_device_ms_policy_counts_' + str(current_time) + '[REDACTED]/.csv',index=False)

In [ ]:
df_windows[df_windows['msPatchPolicy'] == 'No Policy'][['clientName','hostname','msPatchPolicy']].sort_values('clientName',ascending = True).to_csv(export_folder + 'nable_device_missing_patch_policy_' + str(current_time) + '[REDACTED]/.csv',index=False)

In [ ]:
fig = plt.figure(figsize=(30,30))
sns.countplot(data = df_total.sort_values('clientName',ascending=True),y = 'clientName',hue = 'deviceType')

In [ ]:
fig = plt.figure(figsize=(30,30))
sns.countplot(data = df_total.sort_values('clientName',ascending=True),y = 'clientName',hue = 'inIntune')

In [ ]:
fig = plt.figure(figsize=(30,30))
sns.countplot(data = df_total.sort_values('clientName',ascending=True),y = 'clientName',hue = 'inAzureAD')

In [ ]:
fig = plt.figure(figsize=(30, 30))
sns.countplot(data=df_total.sort_values('clientName', ascending=True), y='clientName', hue='agentStatus')

In [ ]:
fig = plt.figure(figsize=(30,30))
sns.countplot(data = df_total.sort_values('clientName',ascending=True),y = 'clientName',hue = 'isWarrantyExpired')

In [ ]:
fig = plt.figure(figsize=(30,30))
sns.countplot(data = df_total.sort_values('clientName',ascending=True),y = 'clientName',hue = 'isWarrantyExpired')

In [ ]:
fig = plt.figure(figsize=(30,30))
sns.countplot(data = df_windows.sort_values('clientName',ascending=True),y = 'clientName',hue = 'msPatchPolicy')